In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from models import iadb
from networks import unet, encoder
import dataset
import gin
import json
import torch

from torch.utils.data import DataLoader
from IPython.display import display, Audio

from tqdm.auto import tqdm

gin.external_configurable(torch.utils.data.DataLoader)

torch.utils.data.dataloader.DataLoader

In [2]:
val_ds = dataset.SLK_VALID_ENCODEC_DB()

Total duration SLK_VAL_ENCODEC_DB: 13:13:20


In [3]:
device = "cuda:0"

path = "../runs/Oct22_14-46-31_vacqueyras.ircam.fr_iadb_decomp_c_runs_Oct22_14-14-19_vacqueyras.ircam.fr_iadb_decomp_c_slakh_db_config_iadb.gin_config_iadb.gin"
cfg_path = [f for f in os.listdir(path) if f.endswith(".gin")]
assert len(cfg_path) == 1

cfg_path = os.path.join(path, cfg_path[0])
chkpt_path = os.path.join(path, "checkpoints.pt")
args_path = os.path.join(path, "args.json")

with open(args_path, "r") as f:
    args = json.load(f)

gin.parse_config_file(cfg_path)

model = iadb.DecompLatentIADB(log=None, device=device, version=args["variant"])

model.load_checkpoint(chkpt_path)
model.models_to_device()
model.set_eval()

dl = DataLoader(val_ds, batch_size=1, shuffle=True, collate_fn=dataset.collate_crop)

for batch in tqdm(dl):
    mix = model.get_x(batch)
    components = model.encoder(mix)
    z = torch.randn_like(mix)
    comp_0 = model.sample(base=z, cond=components, component_idx=0)
    comp_1 = model.sample(base=z, cond=components, component_idx=1)
    mix_hat = model.sample(base=z, cond=components)
    # audio
    x_audio = model.x_to_audio(mix)
    xhat_audio = model.x_to_audio(mix_hat)
    comp_0_audio = model.x_to_audio(comp_0)
    comp_1_audio = model.x_to_audio(comp_1)
    print("Real")
    display(Audio(x_audio.squeeze().cpu().numpy(), rate=model.sr))
    print("Reconstructed")
    display(Audio(xhat_audio.squeeze().cpu().numpy(), rate=model.sr))
    print("Components")
    display(Audio(comp_0_audio.squeeze().cpu().numpy(), rate=model.sr))
    display(Audio(comp_1_audio.squeeze().cpu().numpy(), rate=model.sr))
    break

/data/giovanni/miniconda3/envs/torch23/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Total duration SLK_TRAIN_ENCODEC_DB: 2 days, 18:31:17
Total duration SLK_VAL_ENCODEC_DB: 13:13:20


  0%|          | 0/6800 [00:00<?, ?it/s]

sampling...:   0%|          | 0/50 [00:00<?, ?it/s]

sampling...:   0%|          | 0/50 [00:00<?, ?it/s]

sampling...:   0%|          | 0/50 [00:00<?, ?it/s]

Real


Reconstructed


Components
